In [311]:
from scipy.io import arff
import pandas as pd
import numpy as np
from sklearn import preprocessing
import math
from random import randint
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

In [344]:
data = arff.loadarff('./Data/kc1.arff.txt')
df = pd.DataFrame(data[0])
df['defects'] = df['defects'].apply(lambda x: str(x)[1:]) #removing 'b' from classes
numberOfColumns = len(df.columns) #qtd of columns with defects column
justClass = df[['defects']].values #only the classes values
justData = df.drop(['defects'], axis=1) #removing the classes column for normalization
valuesOfColums = justData.columns.values #get attributes names
justDataValues = justData.values #get data as numpy array

In [345]:
min_max_scaler = preprocessing.MinMaxScaler()
normalized_data = min_max_scaler.fit_transform(justDataValues)
df = pd.DataFrame(normalized_data)
df.columns = valuesOfColums
df.insert(loc=numberOfColumns-1, column='defects', value=justClass)

In [346]:
def euclidian_distance(x, y):
    distance = 0
    for i in range(len(x)-1):
        distance += np.square(x[i] - y[i])
    return np.sqrt(distance)

def calc_dists(propotypes, testInstance):
    d = []
    for p in propotypes:
        d.append([p, euclidian_distance(testInstance, p)])
    d.sort(key=lambda dist: dist[1])
    return d

In [347]:
def lvq1(trainSet, prototypes, lrate, epochs):
    for epoch in range(epochs):
        rate = lrate * (1.0 - (epoch / float(epochs)))
        sum_error = 0
        for example in trainSet:
            neighbor = calc_dists(prototypes, example)[0][0]
            error = np.subtract(example[:-1], neighbor[:-1])
            sum_error += np.sum(np.square(error))
            if(example[-1] == neighbor[-1]):
                neighbor[:-1] = np.add(neighbor[:-1], np.dot(error, rate))
            else:
                neighbor[:-1] = np.subtract(neighbor[:-1], np.dot(error, rate))
    return prototypes

In [348]:
def window(neighbor1, neighbor2, instance, w):
    di = euclidian_distance(neighbor1, instance)
    dj = euclidian_distance(neighbor2, instance)
    if di != 0 and dj != 0:
        mini = min(di/dj, dj/di)
    else:
        mini = 0
    s = ((1-w)/(1+w))
    return (mini > s)

In [349]:
def lvq2_1(trainSet, prototypes, lrate, epochs):
    prots = lvq1(trainSet, prototypes, lrate, epochs)
    for epoch in range(epochs):
        rate = lrate * (1.0 - (epoch / float(epochs)))
        sum_error = 0
        for example in trainSet:
            neighbors = calc_dists(prototypes, example)
            ei, ej = neighbors[0][0], neighbors[1][0]
            if window(ei, ej, example, 0.3):
                if(ei[-1] != ej[-1]):
                    if(ei[-1] == example[-1]):
                        error_i = np.subtract(example[:-1], ei[:-1])
                        error_j = np.subtract(example[:-1], ej[:-1])
                        sum_error += np.sum(np.square(error_i))
                        ei[:-1] = np.add(ei[:-1], np.dot(error_i, rate))
                        ej[:-1] = np.subtract(ej[:-1], np.dot(error_j, rate))
                    elif(ej[-1] == example[-1]):
                        error_i = np.subtract(example[:-1], ei[:-1])
                        error_j = np.subtract(example[:-1], ej[:-1])
                        sum_error += np.sum(np.square(error_j))
                        ej[:-1] = np.add(ej[:-1], np.dot(error_j, rate))
                        ei[:-1] = np.subtract(ei[:-1], np.dot(error_i, rate))
    return prototypes

In [350]:
def lvq3(trainSet, prototypes, lrate, epochs):
    prots = lvq1(trainSet, prototypes, lrate, epochs)
    for epoch in range(epochs):
        rate = lrate * (1.0 - (epoch / float(epochs)))
        sum_error = 0
        for example in trainSet:
            neighbors = calc_dists(prototypes, example)
            ei, ej = neighbors[0][0], neighbors[1][0]
            if window(ei, ej, example, 0.3):
                if(ei[-1] != ej[-1]):
                    if(ei[-1] == example[-1]):
                        error_i = np.subtract(example[:-1], ei[:-1])
                        error_j = np.subtract(example[:-1], ej[:-1])
                        sum_error += np.sum(np.square(error_i))
                        ei[:-1] = np.add(ei[:-1], np.dot(error_i, rate))
                        ej[:-1] = np.subtract(ej[:-1], np.dot(error_j, rate))
                    elif(ej[-1] == example[-1]):
                        error_i = np.subtract(example[:-1], ei[:-1])
                        error_j = np.subtract(example[:-1], ej[:-1])
                        sum_error += np.sum(np.square(error_j))
                        ej[:-1] = np.add(ej[:-1], np.dot(error_j, rate))
                        ei[:-1] = np.subtract(ei[:-1], np.dot(error_i, rate))
                elif (ei[-1] == ej[-1]) and (ei[-1] == example[-1]):
                    error_i = np.subtract(example[:-1], ei[:-1])
                    error_j = np.subtract(example[:-1], ej[:-1])
                    sum_error += np.sum(np.square(np.add(error_i, error_j)))
                    ei[:-1] = np.add(ei[:-1], np.dot(error_i, (rate * 0.3)))
                    ej[:-1] = np.add(ej[:-1], np.dot(error_j, (rate * 0.3)))
    return prototypes

In [351]:
def splitData(train, test):
    trainSet = []
    resultTrainSet = []
    testSet = []
    resultTestSet = []

    for item in train:
        trainSet.append(item[:-1])
        resultTrainSet.append(item[-1])
    for item in test:
        testSet.append(item[:-1])
        resultTestSet.append(item[-1])
    return trainSet, resultTrainSet, testSet, resultTestSet

In [352]:
#CONSTANTS
numberOfPrototypes = 60 #number of prototypes
epochs = 10 #number of epochs training
kfoldNumber = 5
lvqs = [lvq1, lvq2_1, lvq3]

In [353]:
def generatePrototypes(trainSet, numberOfClasses, classesValues):
    prototypes = [[] for i in range(numberOfClasses)]
    proportions = []
    final_prototypes = []
    for i,item in enumerate(classesValues):
        prototypes[i] = [x for x in trainSet if x[-1] == item]
        elements_size = len(prototypes[i])
        proportions.append([math.ceil((elements_size / len(trainSet)) * numberOfPrototypes), item])
    proportions.sort()
    for i,item in enumerate(classesValues):
        size = [x[0] for x in proportions if x[1] == item]
        for j in range(size[0]):
            rand = randint(0, len(prototypes[i])-1)
            final_prototypes.append(prototypes[i][rand])
    return final_prototypes

In [354]:
dataset = np.array(df.values)
Y = df.iloc[:, -1]
numberOfClasses = len(set(Y))
classesValues = list(set(Y))
count = 1
for lvq in lvqs:
    print("LVQ", str(count))
    skf = StratifiedKFold(n_splits=kfoldNumber, shuffle=True, random_state=1)
    totalAccuracy = []
    for train_index, test_index in skf.split(dataset, Y):
        X_train, X_test = dataset[train_index], dataset[test_index]
        prototypeSet = generatePrototypes(X_train, numberOfClasses, classesValues)
        start = time.time()
        LVQ_Prototypes = lvq(X_train, prototypeSet, 0.3, epochs)
        final = time.time() - start
        print("Training time = %.2f" % (final))
        train_data, train_classes, test_data, test_classes = splitData(LVQ_Prototypes, X_test)
        for k in [1, 3]:
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(train_data, train_classes)
            predictions = knn.predict(test_data)
            acc = np.sum([1 for i, x in enumerate(predictions) if x == test_classes[i]])
            totalAccuracy.append([k, (acc / len(predictions))])
    count += 1
    print('K = 1, Accuracy = %.2f' % (sum([x[1] for x in totalAccuracy if x[0] == 1]) / kfoldNumber))
    print('K = 3, Accuracy = %.2f' % (sum([x[1] for x in totalAccuracy if x[0] == 3]) / kfoldNumber))

LVQ 1
Training time = 36.03
Training time = 36.25
Training time = 36.43
Training time = 35.39
Training time = 35.61
K = 1, Accuracy = 0.85
K = 3, Accuracy = 0.85
LVQ 2
Training time = 71.59
Training time = 72.09
Training time = 72.35
Training time = 71.68
Training time = 71.81
K = 1, Accuracy = 0.85
K = 3, Accuracy = 0.85
LVQ 3
Training time = 72.39
Training time = 72.88
Training time = 73.48
Training time = 72.71
Training time = 72.25
K = 1, Accuracy = 0.85
K = 3, Accuracy = 0.85
